In [ ]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm as tqdm
from torch.autograd import Variable
import os
import matplotlib.pyplot as plt
import seaborn as sns
from torch.nn.functional import one_hot
import numpy as np
import pandas as pd
import tqdm
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime as dt
import torchvision
import warnings
import sklearn.metrics as metrics
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import time
from torch.utils.data import TensorDataset, DataLoader
from skimage.io import imread
from skimage.transform import resize
from skimage.color import gray2rgb
from skimage.color import rgba2rgb
from skimage.color import rgb2gray
from torch.utils.data import WeightedRandomSampler
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from PIL import Image
warnings.filterwarnings("ignore")

In [ ]:
!pip install gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1zBEGAP4lCkPBgV7gZePrFGWWTARuXBJD
!gdown https://drive.google.com/uc?id=1kA270AM9_B9snMag4SOsZMvg4syus_EO
!gdown https://drive.google.com/uc?id=1j3VHBoKpNLM-CnNWR2fu5-LZzqN7PDsh
!gdown https://drive.google.com/uc?id=1NfmD1UmEOu5Xzf9CwBRMvsg_1EadFcVt
!gdown https://drive.google.com/uc?id=1vWiWrbtConePYPmQatVG8MVx9t7yENhq
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download jtiptj/chest-xray-pneumoniacovid19tuberculosis --force
!unzip -n chest-xray-pneumoniacovid19tuberculosis
!unzip -n test-20230507T121545Z-001.zip
!unzip -n valid-20230507T121239Z-001.zip
!unzip -n train1.zip
!unzip -n train2.zip


In [ ]:
class Classes_Dataset(TensorDataset):
  def __init__(self, folder_path, orig_path, size = (224,224),  start_idx=None, end_idx=None, return_tensor=True):
    self.folder_path = folder_path.replace("content", "kaggle/working")
    self.orig_path = orig_path.replace("content", "kaggle/working")
    self.normal_path = os.path.join(self.orig_path, 'NORMAL')
    self.tub_path = os.path.join(self.orig_path, 'TURBERCULOSIS')
    self.pneumo_path = os.path.join(self.orig_path, 'PNEUMONIA')
    self.covid_path = os.path.join(self.orig_path,'COVID19')
    self.normal_imgs_len = len(os.listdir(self.normal_path))
    self.tub_imgs_len = len(os.listdir(self.tub_path))
    self.pneumo_imgs_len = len(os.listdir(self.pneumo_path))
    self.covid_imgs_len = len(os.listdir(self.covid_path))
    self.exist = []
    
  
  def __getitem__(self, idx):
    img = torch.load(os.path.join(self.folder_path, 'tensor' + str(idx) + '.pt'))
    label = torch.tensor([0, 0, 0, 0])
    if idx < self.normal_imgs_len:
      label[0] = 1
    elif idx < self.tub_imgs_len + self.normal_imgs_len:
      label[1] = 1
    elif idx < self.pneumo_imgs_len + self.tub_imgs_len + self.normal_imgs_len:
      label[2] = 1
    elif idx < self.covid_imgs_len + self.pneumo_imgs_len + self.tub_imgs_len + self.normal_imgs_len:
      label[3] = 1
    return img, label


  def __len__(self):
    return self.covid_imgs_len + self.pneumo_imgs_len + self.tub_imgs_len + self.normal_imgs_len
    
  def get_label(self, idx):
    label = torch.tensor([0, 0, 0, 0])
    if idx < self.normal_imgs_len:
      label[0] = 1
    elif idx < self.tub_imgs_len + self.normal_imgs_len:
      label[1] = 1
    elif idx < self.pneumo_imgs_len + self.tub_imgs_len + self.normal_imgs_len:
      label[2] = 1
    elif idx < self.covid_imgs_len + self.pneumo_imgs_len + self.tub_imgs_len + self.normal_imgs_len:
      label[3] = 1
    return label

In [ ]:
class Classes_Dataset1(TensorDataset):
  def __init__(self, folder_path, size = (224,224),  start_idx=None, end_idx=None, return_tensor=True):
    print(folder_path, size)
    self.folder_path = folder_path#.replace("content", "kaggle/working")
    self.size = size
    
    self.normal_path = os.path.join(self.folder_path, 'NORMAL')
    self.tub_path = os.path.join(self.folder_path, 'TURBERCULOSIS')
    self.pneumo_path = os.path.join(self.folder_path, 'PNEUMONIA')
    self.covid_path = os.path.join(self.folder_path,'COVID19')
    self.normal_imgs_len = len(os.listdir(self.normal_path))
    self.tub_imgs_len = len(os.listdir(self.tub_path))
    self.pneumo_imgs_len = len(os.listdir(self.pneumo_path))
    self.covid_imgs_len = len(os.listdir(self.covid_path))
    self.exist_images_names = [
        img_folder
        for img_folder in os.listdir(self.normal_path)
    ]

    self.exist_images_names += [
        img_folder
        for img_folder in os.listdir(self.tub_path)
    ]

    self.exist_images_names += [
        img_folder
        for img_folder in os.listdir(self.pneumo_path)
    ]

    self.exist_images_names += [
        img_folder
        for img_folder in os.listdir(self.covid_path)
    ]

    self.return_tensor = return_tensor
  
  def __getitem__(self, idx):
    label = 0
    if idx < self.normal_imgs_len:
      idx_name = self.exist_images_names[idx]
      idx_folder_path = os.path.join(self.normal_path, idx_name)
      label = torch.tensor([1,0,0,0])
    elif idx < self.tub_imgs_len + self.normal_imgs_len:
      idx_name = self.exist_images_names[idx]
      idx_folder_path = os.path.join(self.tub_path, idx_name)
      label = torch.tensor([0,1,0,0])
    elif idx < self.pneumo_imgs_len + self.tub_imgs_len + self.normal_imgs_len:
      idx_name = self.exist_images_names[idx]
      idx_folder_path = os.path.join(self.pneumo_path, idx_name)
      label = torch.tensor([0,0,1,0])
    elif idx < self.covid_imgs_len + self.pneumo_imgs_len + self.tub_imgs_len + self.normal_imgs_len:
      idx_name = self.exist_images_names[idx]
      idx_folder_path = os.path.join(self.covid_path, idx_name)
      label = torch.tensor([0,0,0,1])

    image_obj = imread(idx_folder_path)
    if(len(image_obj.shape) == 3 and image_obj.shape[-1] == 4):
      image_obj = rgba2rgb(image_obj)
    if(len(image_obj.shape) == 3):
      image_obj = rgb2gray(image_obj)
    image_obj = gray2rgb(image_obj)
    if not self.return_tensor:
      return image_obj, label
    else:
      transform = torchvision.transforms.Compose(
    [torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
     ])
      image_obj = resize(image_obj, self.size, mode='constant', anti_aliasing=True,)
      return torch.transpose(torch.transpose(torch.from_numpy(image_obj), 0, 2), 1,2), label

  def __len__(self):
    return len(self.exist_images_names)

In [ ]:
def imshow(img):
    plt.imshow(img.permute(1, 2, 0))
    plt.show()

In [ ]:
train_dataset = Classes_Dataset('/content/train', '/content/train', (224,224))
valid_dataset = Classes_Dataset('/content/valid','/content/val', (224,224))
test_dataset = Classes_Dataset('/content/test', '/content/test', (224,224))

BATCH_SIZE = 16
LR = 0.001
VALID_PART=0.15
TRAIN_PART =0.75
MODEL_NAME = 'ChestModel_v1'
THRESHOLD = 0.5

train_weights = []
for i in range(len(train_dataset)):
    if train_dataset.get_label(i)[0]:
        train_weights.append(0.17)
    elif train_dataset.get_label(i)[1]:
        train_weights.append(0.29)
    elif train_dataset.get_label(i)[2]:
        train_weights.append(0.05)
    elif train_dataset.get_label(i)[3]:
        train_weights.append(0.5)

weighted_sampler = WeightedRandomSampler(
    weights=train_weights,
    num_samples=6000,
    replacement=True
)


os.makedirs(MODEL_NAME, exist_ok=True)

start_time_str = dt.now().strftime('%Y%m%d_%H%M%S')

tb_writer = SummaryWriter(f'runs/{MODEL_NAME}_{start_time_str}')

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=weighted_sampler)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

for images, labels in train_dataloader:
  print(labels)
  imshow(torchvision.utils.make_grid(images))
  break

In [ ]:
def imshow(img):
    plt.imshow(img.permute(1, 2, 0))
    plt.show()

In [ ]:
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
resnet50.eval()
model=resnet50

In [ ]:
model.eval()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = model.double()
model = model.to(device)
model

In [ ]:
model.classifier.fc = nn.Sequential(
    nn.AvgPool2d()
)
# for p in model.layers[:-3]:
#     p.requires_grad = False

# for p in model.layers[-3:]:
#     p.requires_grad = True

# for p in model.fc.parameters():
#     p.requires_grad = True

In [ ]:
import json
from tqdm.notebook import tqdm as tqdm_notebook

In [ ]:
def train(model, train_dataloader, val_dataloader, optimizer, loss_fn, tb_writer, epochs, plot_every = 10, best = 0, save_all = True):
    losses = []
    train_scores = {}
    val_scores = {}
    for epoch in trange(epochs):
        print(f"Epoch {epoch + 1}/{epochs}:")
        train_res = train_one_epoch(model, train_dataloader, optimizer, loss_fn, epoch, tb_writer, plot_every)
        for k, v in train_res.items():
            if train_scores.get(k) is None: train_scores[k] = np.array([])
            train_scores[k] = np.append(train_scores[k], v)
        val_res = val_one_time(model, val_dataloader, loss_fn, epoch)
        for k, v in val_res.items():
            if val_scores.get(k) is None: val_scores[k] = np.array([])
            val_scores[k] = np.append(val_scores[k], v)
        plt.figure(figsize=(25, 5))
        for i, (k, v) in enumerate(val_scores.items()):
            plt.subplot(1, len(val_scores.keys()), i+1)
            plt.plot(np.arange(1, epoch+2), train_scores[k], color='blue', label='Train')
            plt.plot(np.arange(1, epoch+2), val_scores[k], color='orange', label='Val')
            plt.title(k)
            plt.ylabel(k)
            plt.xlabel("epochs")
            plt.legend()
        plt.savefig(os.path.join(MODEL_NAME, f"Results_{start_time_str}"))
        plt.show()
        print(f"train: {make_res_pretty(train_res)}\nval: {make_res_pretty(val_res)}")
        if val_res["recall_macro"] > best or save_all:
            best = val_res["recall_macro"]
            folder_path = os.path.join(MODEL_NAME, f'Model_{epoch + 1}_{start_time_str}')
            os.makedirs(folder_path, exist_ok=True)
            model_path = os.path.join(folder_path, f'model_{epoch + 1}_{start_time_str}')
            torch.save(model.cpu().state_dict(), model_path)
            with open(os.path.join(folder_path, f'modelResults_{epoch + 1}_{start_time_str}.json'), "w") as file:
                json.dump({"train": train_res, "val": val_res}, file)
            print("MODEL SAVED!")
    return val_res['accuracy'], best

In [ ]:
def make_res_pretty(res):
    return '; '.join([f'{k}: {v:.10%}' if k != "avg_loss" else f'{k}: {v}' for k, v in res.items()])

In [ ]:
def train_one_epoch(model, train_dataloader, optimizer, loss_fn, epoch_num, tb_writer, plot_every):
    model.train(True)
    current_loss = 0
    losses = preds = ytrue = np.array([])
    ind = 0
    model = model.to(device)
    for data, labels in tqdm_notebook(train_dataloader):
        data = data.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        yhat = model.forward(data)
        preds = np.append(preds, np.argmax(yhat.cpu().detach().numpy(), axis=1))
        ytrue = np.append(ytrue, np.argmax(labels.cpu().numpy(), axis=1))

        labels=labels.double()

        loss = loss_fn(yhat, labels)

        loss.backward()

        optimizer.step()

        current_loss += loss.item()
        losses = np.append(losses, loss.item())

        if ind % plot_every == 0:
            tb_x = epoch_num * len(train_dataloader) + ind + 1

            tb_writer.add_scalar('Loss/train', loss.item(), tb_x)
        ind += 1

    model.train(False)
    print(preds.shape)
    #preds = np.argmax(preds, axis=1)
    sick = len(preds != 0)
    return  {"avg_loss": current_loss / len(train_dataloader), "accuracy": metrics.accuracy_score(ytrue, preds), 
    "f1_micro": metrics.f1_score(ytrue, preds, average="micro"), "f1_macro": metrics.f1_score(ytrue, preds, average="macro"), 
    "precision_micro": metrics.precision_score(ytrue, preds, average="micro"), "precision_macro": metrics.precision_score(ytrue, preds, average="macro"),
    "recall_micro": metrics.recall_score(ytrue, preds, average="micro"), "recall_macro": metrics.recall_score(ytrue, preds, average="macro"),
    "ill score": sick / len(train_dataloader)}

In [ ]:
def val_one_time(model, valid_dataloader, loss_fn, epoch_number):
    model.eval()  # testing mode
    avg_loss = 0
    preds = ytrue = np.array([])
    model = model.to(device)
    with torch.no_grad():
        for X_val, Y_val in tqdm_notebook(valid_dataloader):
            print(1)
            Y_val = Y_val.to(device)
            X_val = X_val.to(device)

            yhat = model.forward(X_val)

            preds = np.append(preds, np.argmax(yhat.cpu().detach().numpy(), axis=1))
            ytrue = np.append(ytrue, np.argmax(Y_val.cpu().numpy(), axis=1))


            Y_val = Y_val.double()

            avg_loss += loss_fn(yhat, Y_val).item()

    sick = len(preds != 0)
    tb_writer.add_scalar('Loss/valid', avg_loss / len(valid_dataloader), epoch_number)
    return {"avg_loss": avg_loss / len(valid_dataloader), "accuracy": metrics.accuracy_score(ytrue, preds), 
    "f1_micro": metrics.f1_score(ytrue, preds, average="micro"), "f1_macro": metrics.f1_score(ytrue, preds, average="macro"), 
    "precision_micro": metrics.precision_score(ytrue, preds, average="micro"), "precision_macro": metrics.precision_score(ytrue, preds, average="macro"),
    "recall_micro": metrics.recall_score(ytrue, preds, average="micro"), "recall_macro": metrics.recall_score(ytrue, preds, average="macro"),
    "ill score": sick / len(valid_dataloader)}

In [ ]:
max_epochs = 10
bcelosses, bcescore = train(
    model, train_dataloader, valid_dataloader,
    optimizer, loss_fn, tb_writer, max_epochs)

In [ ]:
os.listdir(r'/kaggle/working/ChestModel_v1/Model_3_20230507_152046')

In [ ]:
os.chdir(r'/kaggle/working/')

!tar -czf model_3_20230507_152046 'ChestModel_v1/Model_3_20230507_152046'

from IPython.display import FileLink 

FileLink(r'model_3_20230507_152046')

In [ ]:
!zip -r model_3_20230507_152046.zip r'/kaggle/working/ChestModel_v1/Model_3_20230507_152046'

In [ ]:
!zip -r model_3_20230507_152046.zip . -i r/kaggle/working/ChestModel_v1/Model_3_20230507_152046

In [ ]:
train(model, loss_fn, optimizer, n_epoch=10, device=device)

In [ ]:
model.to(torch.device('cpu'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
torch.save(model.state_dict(), 'content')

In [ ]:
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b4', pretrained=True)#model = resnet50(pretrained=True)]


In [ ]:
model

In [ ]:
model.fc=nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(2048, 4),
    nn.Softmax()
)

In [ ]:
model = model.double()
model = model.eval()

In [ ]:
state_dict = torch.load("/kaggle/working/Efficientnet_b4_model_8_20230507_181213", map_location=torch.device('cpu'))


In [ ]:
model.load_state_dict(state_dict)
model = model.eval()

In [ ]:
device

In [ ]:
model = model.cpu()

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
val_one_time(model, test_dataloader, loss_fn, epoch_number=1)